# **Procesamiento del habla**

**VERSION 1.0**

## Funcionalidades desbloqueadas por versión:

Ver 1.0
- Uso de dataset filtrado por frutas y verduras
- Entrada de consulta para un producto por vez
- Recomendación para el cliente:
    - Algoritmo de ordenamiento basado en preferencias del usuario (dentro de la estructura gramatical):
        - Basado en el precio.
        - Características particulares del producto.

*Ver 1.3:*
- Entrada de consulta para más de un producto.
- Recomendaciones basadas en análisis de sentimiento.


## Estructura de sub-etapas
--- 
```
├── Reconocimiento del lenguaje
│ ├── Captura de texto
│ └── Captura de audio
├── Procesamiento del lenguaje
│ ├── Preprocesamiento de texto
│ └── Análisis de texto
├── Gestión de respuesta
│ ├── Gestión del estado
│ ├── Motor de respuesta
│ └── Personalización
├── Texto a voz
│ └── Conversión de texto a voz

```



Se utilizaran las siguientes principales librerias por etapas, speechrecognition para reconocimiento de voz, pydub para transcripcion a texto, spacy para el procesamiento y analisis de texto y Google Text-to-Speech para la conversion de texto a voz.

Para la entrega de la recomendacion se tomaran los tokens lematizados (filtrados por adjetivos, sustantivos y proposiciones) de la entrada del usuario y de la columna de productos del dataframe, y se recorrera token por token entre cada una, para comprobar la similitud, entregando por ultimo, un diccionario con el indice del producto en la tabla original y el grado de considencia.

### Librerias y paquetes

In [1]:
# detectar las prioridades del usuario por medio de los adjetivos (precio:[economico, barato, buen precio], otros_adjetivos[...])

#### *Entorno virtuales*

In [2]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

#### *Instalacion*

In [3]:
# Librerías de preprocesamiento
!pip3 install sounddevice
!pip3 install scipy
!pip3 install pydub

# Librerías de reconocimiento de voz
!pip3 install pocketsphinx
!pip3 install SpeechRecognition
!pip3 install pyaudio

# Librerías de generación de voz
!pip3 install gtts

# Librerías de análisis lingüístico
!pip3 install spacy
!pip3 install wordnet

  Using cached wordnet-0.0.1b2.tar.gz (8.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      Traceback (most recent call last):
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 118, in get_requires_for_buil

In [4]:
# Descargo el modelo de la libreria spacy pre-entrenado con corpus en Español
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 838.7 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


#### *Importacion*

In [5]:
# Librerías para reconocimiento de voz y procesamiento de audio
import speech_recognition as sr
import sounddevice as sd
from scipy.io.wavfile import write
import queue
import threading
from pydub import AudioSegment
import pyaudio

# Librerías para manipulación de archivos y compresión
import wave
import zipfile
import os
import subprocess
import io

# Librerías para procesamiento de datos y análisis
import conllu
from conllu import parse_incr
import numpy as np
import pandas as pd
import spacy
import re
import nltk
# Descargar datos de WordNet si es necesario
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

# Librería para generación de voz
from gtts import gTTS

# Importación de funciones comunes a otros cuadernos
from funciones_comunes import common_functions

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


### Lectura de datos

#### *Datos fuentes*

In [6]:
# Defino el diccionario donde se almacenara la informacion de entrada y salida de la consulta
consulta_dict = {}

In [7]:
# Lectura y almacenado del dataframe
path_csv_rutas_verduras = '../../datos/procesaodos/VerdurasporSupermercado.csv'
df_frutas_verduras = pd.read_csv(path_csv_rutas_verduras)

## 1. Reconocimiento del Habla

___

### a. Captura de entradas de texto

In [8]:
# Conexión con la API de FastAPI
# Aquí iría la conexión con un script encargado de conectar con la API de FastAPI

In [9]:
# Función tentativa para recibir la entrada de texto desde FastAPI
def entrada_texto(texto):
    texto = input('Ingresa brevemente el producto que deseas comprar y sus características: ')
    return texto

### b. Captura de Audio

#### Reconocimiento del Habla (ASR - Automatic Speech Recognition):

In [10]:
def entrada_voz():
    recognizer = sr.Recognizer()  # Se inicializa el objeto de la clase speech recognition con el metodo recognizer y almacena en la variable 'recognizer'
    mic = sr.Microphone()  # Configuración del micrófono como fuente de audio

    with mic as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)
        # Ajuste de sensibilidad y tiempo de pausa
        recognizer.energy_threshold = 300  # Ajustar según el entorno
        recognizer.pause_threshold = 1.0  # Ajustar según necesidad
        print("Por favor, habla ahora.")  # Indicación para el usuario
        audio = recognizer.listen(source)  # Escucha y captura del audio mientras escuche entrada de voz continua

    try:
        texto = recognizer.recognize_google(audio, language='es-ES')  # Reconocimiento de voz utilizando Google
    except sr.UnknownValueError:
        print("No se pudo entender el audio")  # Manejo de error en caso de audio no reconocido
    except sr.RequestError as e:
        print("Error al conectarse con el servicio de Google: {0}".format(e))  # Manejo de error en la conexión con Google

    return texto  # retorna el texto reconocido por audio

### c. Identificar tipo de entrada

In [11]:
# Tipo de entrada específica pasada por fast-api
entrada_tipo = ''

# ejecucion de funciones de entrada segun corresponda la modalidad / tipo de la misma
if entrada_tipo == 'audio':
    texto = entrada_voz()  # Si la entrada es de tipo 'audio', se utiliza la función para recibir entrada de voz
elif entrada_tipo == 'texto':
    texto = entrada_texto()  # Si la entrada es de tipo 'texto', se utiliza la función para recibir entrada de texto
else:
    pass

## 2. Procesamiento del Lenguaje Natural

___

In [12]:
# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

### Preprocesamiento de texto


In [13]:
# Lista de abreviaturas y formas no abreviadas de unidades de medida de peso
'''

# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',
                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',
                        #'toneladas', 'libras', 'onzas', 'quintales']

'''

"\n\n# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',\n                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',\n                        #'toneladas', 'libras', 'onzas', 'quintales']\n\n"

In [14]:
# Función encargada de procesar texto
def procesar_texto(texto):
    texto = texto.lower()  # convierte el texto a minúsculas
    texto = re.sub(r'[,\.;:!\-*#@$!+_%^&`~]', '', texto)  # Elimina los caracteres especiales
    texto = re.sub(r'\s+', ' ', texto)  # Elimina los espacios en blanco adicionales

    thresgold = 1  # se establece el umbral para la longitud mínima de las palabras

    doc = nlp(texto)  # Se pre-procesa el texto utilizando la clase pre-etrenada de spaCy

    # Se obtienen los tokens, lemas y etiquetas de parte del discurso para las palabras filtradas
    tokens = [token.text for token in doc]
    # se definen la lista de palabras que no aportan
    stopwords = spacy.lang.es.stop_words.STOP_WORDS
    # se filtran los tokens por medio de diversas condiciones
    filtered_tokens = [
        token.text for token in doc # se recorre cada token de la entrada
        if token.text not in stopwords # si comprueba que el token este fuera de un stopword
        and token.pos_ in ('NOUN', 'ADJ', 'ADP') # se comprueba que la etiqueta gramatical del token sea sustantivo, adjetivo o proposicion
        and len(token.text) > thresgold # Se comprueba que la longitud del token sea mayor al umbral previamente establecido
        ]
    lemmas = [token.lemma_ for token in doc if token.text in filtered_tokens]
    pos_tags = [(token.text, token.pos_) for token in doc if token.text in filtered_tokens]

    return {
        "tokens": tokens,
        "lemmas": lemmas,
        "filtered_tokens": filtered_tokens,
        "pos_tags": pos_tags,
    }



**Normalización del texto (eliminación de ruido, corrección ortográfica, etc.)**

In [15]:
texto = 'hola quiero tres papas'

In [16]:
# texto procesado por la funcion 'procesar_texto'
texto_analizado = procesar_texto(texto)

**Tokenización**

In [17]:
# tokens del texto procesado
tokens = texto_analizado['tokens']

**Eliminación de stop words.**


In [18]:
# Tokens filtrados
tokens_filtrados = texto_analizado['filtered_tokens']

**Lematización y stemming**

In [19]:
# lemas de los tokens filtrados
lemmas_filtrados = texto_analizado['lemmas']

Etiquetado de estructuras gramaticales:

In [20]:
# etiquetas gramaticales de los tokens filtrados
pos_tags_filtrados = texto_analizado['pos_tags']

### Análisis de texto


In [21]:
# Palabras de referencia para preferencias basadas en el precio
palabras_referencia = ["económico", "precio"]

In [22]:
'''
def verifica_sinonimo_precios(tokens, referencias):
    '''
    Esta funcion verifica si dentro de la lista de tokens proporcionada como primer parametro corresponde a un sinonimo o derivacion morfologica
    de algunas de las palabras dentro de la lista de referencia.

    Args:
    'tokens': Recibe los tokens lematizados de la entrada del usuario.
    'referencias': Recibe una lista de referencias con las posibles palabras lematizadas, basadas en preferencias del usuario en relación con el precio.

    Retorno:
    Esta funcion retorna como resultado 'True' o 'False', según corresponda.
    '''
    for referencia in referencias:  # recorre la lista 'referencias'
        synsets = wn.synsets(referencia, pos=wn.ADJ) + wn.synsets(referencia, pos=wn.NOUN)  # crea una lista de sinónimos de cada una de las referencias
        for synset in synsets:  # recorre cada uno de los sinónimos de la lista de referencia
            for lemma in synset.lemmas():  # obtiene el lema de cada uno de los sinónimos
                if lemma.name() in tokens:  # verifica si el lema está en los tokens lematizados del usuario
                    return True
                if lemma.derivationally_related_forms():
                    for related_lemma in lemma.derivationally_related_forms():
                        if related_lemma.name() in tokens:  # verifica si el lema derivacional está en los tokens lematizados del usuario
                            return True
    return False

# Lista de referencias con las posibles palabras relacionadas con "económico" y "precio"
referencias = ["económico", "barato", "asequible", "precio", "coste"]

# Verificar si hay algún sinónimo o derivación en los tokens lematizados
consulta_precio = verifica_sinonimo_precios(lemmas_filtrados, referencias)
'''

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/cristianariel/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


Detectar presencia de palabras de productos lematizadas dentro de dataframe

In [ ]:
# carga del conjunto de datos en dataframe
df_frutas_verduras['producto_tokens_lemmas'] = df_frutas_verduras['Producto'].apply(lambda x: procesar_texto(x)['lemmas'])

In [ ]:
# Diccionario de coincidencias entre tokens lematizados de la columna y la entrada del usuario
dict_count_coincidences = {}

In [ ]:
# Algoritmo para contar coincidencias entre tokens lematizados de la entrada del usuario y la columna/fila del DataFrame
for index, row in df_frutas_verduras.iterrows():  # Iterar sobre las filas del DataFrame
    for token_producto in row['producto_tokens_lemmas']:  # Iterar sobre los tokens lematizados de la columna 'producto_tokens_lemmas'
        if token_producto in lemmas_filtrados:  # Verificar si el token lematizado está en la lista de lemas filtrados de la entrada del usuario
            if token_producto not in dict_count_coincidences:  # Si el token no está en el diccionario de coincidencias
                dict_count_coincidences[index] = 1  # Agregar el token al diccionario con el valor 1
            else:  # Si el token ya está en el diccionario de coincidencias
                dict_count_coincidences[index] += 1  # Incrementar el valor del token en el diccionario

In [ ]:
# Se transforma el diccionario a una seria de python para luego poder ordenarla por valor
series_ordered_count_coincidences = pd.Series(dict_count_coincidences)

Análisis de Sentimiento: Determinar la emoción o tono del texto.


In [ ]:
# se desarrollara en futuras versiones

Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


In [ ]:
# se desarrollara en futuras versiones

## 3. Gestión del Diálogo

___

### Módulo de Gestión de Estado:
Llevar un registro del contexto y estado del diálogo para mantener conversaciones coherentes.


In [ ]:
# Asignación de valores al diccionario de consulta
consulta_dict['id_consulta'] = None  # ID de la consulta (aún no asignado)
consulta_dict['id_cliente'] = None  # ID del cliente (aún no asignado)
consulta_dict['formato_consulta'] = None  # Formato de la consulta (aún no especificado)
consulta_dict['transcripción_audio'] = None  # Transcripción del audio (aún no disponible)
consulta_dict['entrada_texto'] = texto  # Texto de la entrada proporcionada por el usuario
consulta_dict['entrada_lemas_filtrados'] = lemmas_filtrados  # Lemas filtrados de la entrada del usuario
consulta_dict['dict_producto_indice_considencias'] = dict_count_coincidences  # Diccionario de productos y sus coincidencias
consulta_dict['card_recomendacion'] = None  # Tarjeta de recomendación (aún no generada)
audio_recomendacion = None  # Audio de la recomendación (aún no generado)

### Motor de Respuesta:


Generación de Respuestas: Utilizar modelos generativos (como GPT-3) o respuestas predefinidas según las intenciones y entidades detectadas.


In [ ]:
# Limpiando la columna 'Precio': eliminando el signo de dólar, eliminando los puntos del millar y reemplazando la coma decimal por un punto
df_frutas_verduras['Precio'] = df_frutas_verduras['Precio'].apply(common_functions.limpiar_signo_peso)

In [ ]:
# Asegurar que series_ordered_count_coincidences es una Serie ordenada
series_ordered_count_coincidences = series_ordered_count_coincidences.sort_values(ascending=False)

# Inicialización de la recomendación
recomendacion = 'Los productos recomendados basados en su consulta y características mencionadas son:\n'

# Lista para almacenar las recomendaciones
lista_string_reco_supers_prods = []

# Iterar sobre los índices de las coincidencias ordenadas
for count, x in enumerate(series_ordered_count_coincidences.index, start=1):
    # Extraer los detalles del producto usando el índice
    producto, supermercado, precio = df_frutas_verduras.loc[x, ['Producto', 'Supermercado', 'Precio']]

    # Añadir la recomendación a la lista
    lista_string_reco_supers_prods.append(f'En la posición {count}: {producto} en {supermercado} a {precio} pesos\n')

# Concatenar todas las recomendaciones en un solo string
recomendacion += ''.join(lista_string_reco_supers_prods)

# Redondear números en la recomendación (si es necesario)
recomendacion = common_functions.redondear_numeros(recomendacion)

# Asignar la recomendación al diccionario de consulta
consulta_dict['card_recomendacion'] = recomendacion

Selección de Respuestas: Elegir la mejor respuesta entre varias opciones generadas.


In [ ]:
# se desarrollara en futuras versiones

### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario y el contexto actual.

In [ ]:
# se desarrollara en futuras versiones

## 4. texto a voz

___

### Conversión de texto a Voz (TTS): Utilizar servicios como Gogle texto-to-Speech, Amazon Polly, o frameworks como Tacotron para convertir el texto generado en voz.


In [ ]:
# google text to speech tomará la variable recomendacion como el texto a convertir en voz y establecerá el idioma como español ('es')
tts = gTTS(text=recomendacion, lang='es')

In [ ]:
# Guardar el audio en un objeto BytesIO
audio_buffer = io.BytesIO()
tts.write_to_fp(audio_buffer)

In [ ]:
# Asegurarse de que el puntero esté al principio del buffer
audio_buffer.seek(0)

0

In [ ]:
# Se guarda el audio de respuesta en el diccionario de la consulta
consulta_dict = {'recomendacion_audio': audio_buffer}